In [1]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict

In [23]:
reviews_raw = open('yelp_academic_dataset_review.json', 'r')

counter = 0

data_dict = defaultdict(list)

for line in reviews_raw:
    if counter >= 3:
        break
    #print line
    line_json = json.loads(line)

    user_id = line_json['user_id']
    review_type = line_json['type']
    stars = line_json['stars']
    date = line_json['date']
    business_id = line_json['business_id']
    text = line_json['text']

    data_dict['user_id'].append(user_id)
    data_dict['stars'].append(stars)
    data_dict['business_id'].append(business_id)
    data_dict['text'].append(text)
    
    #Print a sample 
    #if user_id == 'tAKjY3bQXH51msJbOHYPmQ':
    #    print line_json
    
    #print user_id, business_id, stars
    #counter += 1

#print dict(data_dict)
    
reviews_raw.close() 

df = pd.DataFrame(data_dict)

#keep only 4-5 star reviews

df = df[df['stars'] >= 4]



In [114]:
businesses_raw = open('yelp_academic_dataset_business.json', 'r')

#this table has different fields in different columns 
#extract all the possible values from the data? 
#get all possible values of attributes, ambience, and categories 
#there are multiple levels of nesting... 
#for attributes I need each key and all the possible values (it's not always true/false)


fields = set()
fields_amb = set()
fields_cat = set()
attributes_possible = defaultdict(set)

counter = 1

for line in businesses_raw:
        
    if counter >= 6:
        break
    line_json = json.loads(line)
    
    if 'Restaurants' not in line_json['categories']:
        continue
        
    #print line_json
    
    #attributes
    attributes = line_json['attributes']
    for key in attributes:
        #print key
        current_val = attributes[key]
        attributes_possible[key].add(str(current_val))
        
    #ambience (not always present in attributes so we need a try/except)
    try:
        ambience = line_json['attributes']['Ambience']
        for key in ambience:
            fields_amb.add(key)
    except(KeyError): 
        pass
    
    #counter to only test for a few rows
    #counter += 1
 
    #categories
    categories = line_json['categories']
    for key in categories:
        #print key
        fields_cat.add(key)    

#keep only True/False attributes 


#alcohol
#price range 

for key in attributes_possible:
    if attributes_possible[key] == set(['True', 'False']):
        fields.add(key)

print fields
        
#check out all the possible attributes 
target=open('attributes_possible', 'w')
for key in attributes_possible:
    target.write(key)
    target.write('\t')
    target.write(str(attributes_possible[key]))
    target.write('\n')

set([u'Delivery', u'Take-out', u'Drive-Thru', u'Open 24 Hours', u'Good For Dancing', u'Waiter Service', u'Good For Kids', u'Corkage', u'BYOB', u'Dogs Allowed', u'Happy Hour', u'Order at Counter', u'Has TV', u'Outdoor Seating', u'Coat Check', u'By Appointment Only', u'Takes Reservations', u'Good for Kids', u'Good For Groups', u'Caters', u'Wheelchair Accessible'])


In [126]:

businesses_raw = open('yelp_academic_dataset_business.json', 'r')

counter = 0
#explode the attributes and categories 

business_dict = defaultdict(list)

for line in businesses_raw:
    if counter >= 10:
        break
   
    #print line
    line_json = json.loads(line)
    
    #only keep restaurants
    if 'Restaurants' not in line_json['categories']:
        continue
    
    #only keep businesses with reviews >= 10 
    if line_json['review_count'] < 10:
        continue
    
    #print a sample json
   
    
    #city = line_json['city']
    bus_stars = line_json['stars']
    review_count = line_json['review_count']
    #categories = line_json['categories']
    #attributes = line_json['attributes']
    name = line_json['name']
    business_id = line_json['business_id']
    
    

    #business_dict['city'].append(city)
    business_dict['bus_stars'].append(bus_stars)  
    business_dict['review_count'].append(review_count) 
    #business_dict['categories'].append(categories) 
    #business_dict['attributes'].append(attributes) 
    business_dict['name'].append(name) 
    business_dict['business_id'].append(business_id) 
    
    
    
    try:
        price = line_json['attributes']['Price Range']
        business_dict['price'].append(price) 
    except(KeyError):
        business_dict['price'].append('1')

    
    try:
        alcohol = line_json['attributes']['Alcohol']
        business_dict['alcohol'].append(alcohol) 
    except(KeyError):
        business_dict['alcohol'].append('none')
    
    
    
    if business_id == 'stH6XAn2Drzol1H5oGvL2A':
        target = open('business_sample.txt','w')
        target.write(line)
    
    
    #attributes are not always true/false
    
    #get all the attributes as separate columns
    for attribute in fields:
        if attribute == 'Ambience':
            continue
        try:
            value = line_json['attributes'][attribute]
            if str(value).lower() == 'true':
                value = True
            else:
                value = False
        except(KeyError):
            value = False
        business_dict['atr_' + attribute].append(value) 
    
    #get the ambience out of the attributes
    
    for ambience in fields_amb:
        try:
            value = line_json['attributes']['Ambience'][ambience]
            if str(value).lower() == 'true':
                value = True
            else:
                value = False
        except(KeyError):
            value = False
        business_dict['amb_' + ambience].append(value) 
        
    
    for category in fields_cat:
        if category in line_json['categories']:
            business_dict['cat_' + category].append(True) 
        else:
            business_dict['cat_' + category].append(False) 

businesses_raw.close() 

df_bus = pd.DataFrame(business_dict)


df_bus.to_csv('df_bus_out.csv', encoding='UTF-8')


In [127]:
df_bus.head()

,alcohol,amb_casual,amb_classy,amb_divey,amb_hipster,amb_intimate,amb_romantic,amb_touristy,amb_trendy,amb_upscale,...,cat_Venues & Event Spaces,cat_Vietnamese,cat_Wedding Planning,cat_Wine Bars,cat_Wineries,cat_Wok,cat_Yoga,name,price,review_count
0,full_bar,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,Emil's Lounge,1,11
1,full_bar,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,Alexion's Bar & Grill,1,15
2,none,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,Gab & Eat,1,38
3,full_bar,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,Papa J's,2,46
4,none,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,Steak 'n Shake,1,36


In [128]:
#replace all true/false with 0/1

df_bus.replace(True, 1.0, inplace=True)
df_bus.replace(False, 0.0, inplace=True)
df_bus['alcohol'] = df_bus['alcohol'].map({'none':0, 'beer_and_wine':1, 'full_bar':2}).astype(float)
df_bus['price'] = df_bus['price'].astype(float)
#keep only businesses with reviews > 10 
df_bus = df_bus[df_bus['review_count'] >= 10]
df_bus.head()

,alcohol,amb_casual,amb_classy,amb_divey,amb_hipster,amb_intimate,amb_romantic,amb_touristy,amb_trendy,amb_upscale,...,cat_Venues & Event Spaces,cat_Vietnamese,cat_Wedding Planning,cat_Wine Bars,cat_Wineries,cat_Wok,cat_Yoga,name,price,review_count
0,2,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,Emil's Lounge,1,11
1,2,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,Alexion's Bar & Grill,1,15
2,0,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,Gab & Eat,1,38
3,2,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,Papa J's,2,46
4,0,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,Steak 'n Shake,1,36


In [149]:
#join with the user review data 
#merge, filter and sort 
 
df_merged = pd.merge(df, df_bus, on='business_id')
#keep only users with 10+ reviews
#groupby.filter can do this i think
df_merged = df_merged.groupby('user_id', as_index=False).filter(lambda x: len(x) >= 10)
#df_merged.sort('user_id', inplace=True)




In [150]:
#remove some non numeric columns 
df_merged.drop(['stars','bus_stars','text', 'name'], axis=1, inplace=True)

#there's an issue - one user might be reviewing the same business multiple times 
#try duplicates drop 
print df_merged.info()
df_merged.drop_duplicates(take_last=True, subset=['user_id', 'business_id'], inplace=True)
print df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 238686 entries, 2 to 632236
Columns: 319 entries, business_id to review_count
dtypes: bool(314), float64(2), int64(1), object(2)
memory usage: 82.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 226566 entries, 2 to 632236
Columns: 319 entries, business_id to review_count
dtypes: bool(314), float64(2), int64(1), object(2)
memory usage: 78.2+ MB
None


In [151]:
#change to numeric 
df_merged.replace(True, 1.0, inplace=True)
df_merged.replace(False, 0.0, inplace=True)


In [153]:

df_merged['review_count'] = 1
df_merged.head()

,business_id,user_id,alcohol,amb_casual,amb_classy,amb_divey,amb_hipster,amb_intimate,amb_romantic,amb_touristy,...,cat_Venezuelan,cat_Venues & Event Spaces,cat_Vietnamese,cat_Wedding Planning,cat_Wine Bars,cat_Wineries,cat_Wok,cat_Yoga,price,review_count
2,mVHrayjG3uZ_RLHkLj-AMg,8fApIAMHn2MZJFUiCQto5Q,2,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,1,1
8,mVHrayjG3uZ_RLHkLj-AMg,tAKjY3bQXH51msJbOHYPmQ,2,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,1,1
11,KayYbHCt-RkbGcPdGOThNg,JbWnXwjLaO1kncUkQOMajg,2,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,1,1
12,KayYbHCt-RkbGcPdGOThNg,4-3IU5uUH90m21TWbZhhnA,2,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,1,1
18,b9WZJp5L1RZr4F1nxclOoQ,FNbm3ycU2BF8C17UFfWzOg,0,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,1,1


In [154]:
grouped_metrics = df_merged.groupby('user_id', as_index=False).agg(np.sum)


In [161]:
#Start combining the columns to the user level 
#Use the term frequency divided by the users's review count


grouped_metrics.head()
#divide each column by the review count 
grouped_metrics.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10107 entries, 0 to 10106
Data columns (total 318 columns):
user_id                                 object
alcohol                                 float64
amb_casual                              float64
amb_classy                              float64
amb_divey                               float64
amb_hipster                             float64
amb_intimate                            float64
amb_romantic                            float64
amb_touristy                            float64
amb_trendy                              float64
amb_upscale                             float64
atr_BYOB                                float64
atr_By Appointment Only                 float64
atr_Caters                              float64
atr_Coat Check                          float64
atr_Corkage                             float64
atr_Delivery                            float64
atr_Dogs Allowed                        float64
atr_Drive-Thru             

In [162]:
#change this to be dividing all the columns I care about by the last review count column 

divided = grouped_metrics[grouped_metrics.columns[1:317]].divide(grouped_metrics[grouped_metrics.columns[-1]], axis = 0)

#merge the userid back in

userids = grouped_metrics['user_id']


divided_user = pd.concat([userids, divided], axis=1)
divided_user.head()
divided_user.to_csv('final_dataset.csv', index=False)

In [163]:
divided_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10107 entries, 0 to 10106
Columns: 317 entries, user_id to price
dtypes: float64(316), object(1)
memory usage: 24.5+ MB


In [165]:
divided_user.head()

,user_id,alcohol,amb_casual,amb_classy,amb_divey,amb_hipster,amb_intimate,amb_romantic,amb_touristy,amb_trendy,...,cat_Vegetarian,cat_Venezuelan,cat_Venues & Event Spaces,cat_Vietnamese,cat_Wedding Planning,cat_Wine Bars,cat_Wineries,cat_Wok,cat_Yoga,price
0,--65q1FpAL_UQtVZ2PTGew,0.795918,0.765306,0.020408,0.091837,0.020408,0.020408,0.010204,0,0.020408,...,0.000000,0,0,0.061224,0,0.010204,0,0,0,1.448980
1,--VxRvXk3b8FwsSbC2Zpxw,1.200000,0.733333,0.066667,0.066667,0.000000,0.066667,0.000000,0,0.066667,...,0.000000,0,0,0.000000,0,0.000000,0,0,0,1.733333
2,-0itF0VWVBe3k2AdfUReGA,0.884615,0.461538,0.038462,0.076923,0.076923,0.076923,0.076923,0,0.153846,...,0.000000,0,0,0.038462,0,0.000000,0,0,0,1.692308
3,-0wUMy3vgInUD4S6KJInnw,0.916667,0.666667,0.083333,0.083333,0.083333,0.000000,0.000000,0,0.083333,...,0.083333,0,0,0.000000,0,0.000000,0,0,0,1.750000
4,-2EuoueswhqEERWezJY8gw,1.470588,0.470588,0.058824,0.058824,0.117647,0.000000,0.117647,0,0.176471,...,0.000000,0,0,0.000000,0,0.000000,0,0,0,2.000000
